In [35]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor

In [26]:
data = pd.read_csv('201901-202003.csv')
data = data.fillna('')

In [27]:
submission = pd.read_csv('submission.csv', index_col=0)

In [28]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [29]:
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [30]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [31]:
train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = train_num['CNT'] / 2134015
z = train_num['CSTMR_CNT'] / 649192

In [90]:
train_num = df_num.sample(frac=1, random_state=0)
train_num
train_num['gap']= train_num['CNT'] - train_num['CSTMR_CNT']
train_num.loc[train_num['gap'] <0,'mark'] = '취소있음'
train_num.loc[train_num['gap'] ==0,'mark'] = '고객다름'
train_num.loc[train_num['gap'] >0,'mark'] = '단골있음'

In [32]:
two_Y = np.zeros((len(y),2))
two_Y[:,0] = y
two_Y[:,1] = z

In [73]:
import lightgbm as lgb
from tqdm import tqdm
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split( x, two_Y, 
#                                      test_size=0.2, random_state=0)
#train_ds = lgb.Dataset(X_train, label=y_train)
#val_ds = lgb.Dataset(X_test, label=y_test)

y_pred =0
N =2
for i in tqdm(range(N)):
    model = MultiOutputRegressor(lgb.LGBMRegressor(random_state=i*101,max_depth= -1,
            learning_rate = 0.05,
            boosting_type='gbdt',
            objective= 'regression',
            metric= 'mae',
            num_boost_round=5000,
            sub_row = 0.75,
            lambda_l2 = 0.1,nfold=5), n_jobs=-1)
    model.fit(x, two_Y , 
    )
    y_pred += model.predict(temp.iloc[:,:-2])
y_pred /= N


100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [59:11<00:00, 1775.63s/it]


In [77]:
temp['CSTMR_CNT'] = np.clip(np.exp(y_pred[:,1])-1, 0, None)
temp['CNT']=np.clip(np.exp(y_pred[:,0])-1, 0, None)

In [80]:
display(temp)
display(train_num)
train_num['CSTMR_CNT'] = train_num['CSTMR_CNT'] / 649192
train_num['CNT'] = train_num['CNT'] / 2134015

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CNT,CSTMR_CNT
0,0,0,0,1,1,1,2020,4,0.000000,0.000000
1,0,0,0,1,1,1,2020,7,0.000000,0.000000
2,0,0,0,1,1,2,2020,4,0.000000,0.000366
3,0,0,0,1,1,2,2020,7,0.000000,0.000302
4,0,0,0,1,1,3,2020,4,0.000644,0.003909
...,...,...,...,...,...,...,...,...,...,...
1658855,16,30,14,0,2,3,2020,7,0.000000,0.000000
1658856,16,30,14,0,2,4,2020,4,0.000000,0.000000
1658857,16,30,14,0,2,4,2020,7,0.000000,0.000000
1658858,16,30,14,0,2,5,2020,4,0.000000,0.000000


,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
149245,2,4,2,2,1,2,2019,4,5,275000,5
554956,8,26,11,1,1,2,2020,2,3,89300,6
918516,14,33,14,4,2,4,2019,5,10262,795920911,15027
425414,7,18,8,2,2,1,2019,10,120,2814110,164
640949,9,39,16,5,2,5,2020,1,9,349000,9
...,...,...,...,...,...,...,...,...,...,...,...
359783,6,16,9,1,1,1,2019,12,382,3756020,530
152315,2,7,11,3,2,3,2020,3,3,229910,5
963395,15,19,9,5,1,5,2020,2,13,883000,15
117952,1,31,4,1,1,2,2019,7,3,107440,6


In [81]:
train_num['gap']= train_num['CNT'] - train_num['CSTMR_CNT']
temp['gap']= temp['CNT'] - temp['CSTMR_CNT']
train_num.loc[train_num['gap'] <0,'mark'] = '취소있음'
train_num.loc[train_num['gap'] ==0,'mark'] = '고객다름'
train_num.loc[train_num['gap'] >0,'mark'] = '단골있음'
temp.loc[temp['gap'] <0,'mark'] = '취소있음'
temp.loc[temp['gap'] ==0,'mark'] = '고객다름'
temp.loc[temp['gap'] >0,'mark'] = '단골있음'

In [96]:
# 인코딩
dtypes = train_num.dtypes
encoders = {}
for column in train_num.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(train_num[column])
        encoders[column] = encoder
        
train_num = train_num.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    train_num[column] = encoder.transform(train_num[column])

# 인코딩
dtypes = temp.dtypes
encoders = {}
for column in temp.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(temp[column])
        encoders[column] = encoder
        
temp_num = temp.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    temp_num[column] = encoder.transform(temp[column])

In [126]:
train_num=train_num.drop(['gap'], axis=1)
temp_num=temp_num.drop(['gap'], axis=1)

In [128]:
train_num['CSTMR_CNT'] = train_num['CSTMR_CNT'] / 649192
train_num['CNT'] = train_num['CNT'] / 2134015
train_num.mark.unique()
X = train_num.drop(['AMT'], axis=1)
Y = np.log1p(train_num['AMT']) 

In [129]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, Y, 
                                      test_size=0.2, random_state=0)
train_ds = lgb.Dataset(X_train, label=y_train)
val_ds = lgb.Dataset(X_test, label=y_test)
def rmsle_1(y_pred, data):
    y_true = np.array(data.get_label())
    score= np.sqrt(np.square(np.log1p(y_pred + 1) - np.log1p(y_true + 1)).mean())
    return 'rmsle', score, False
params = {
            'max_depth': -1,
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'tweedie_variance_power': 1.1,
            'metric': 'custom',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }
model = lgb.train(params,
                  train_ds,
                  5000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100,
                  feval=rmsle_1
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmsle: 0.036747
[200]	valid_0's rmsle: 0.0333253
[300]	valid_0's rmsle: 0.0322404
[400]	valid_0's rmsle: 0.0315622
[500]	valid_0's rmsle: 0.0310717
[600]	valid_0's rmsle: 0.0306908
[700]	valid_0's rmsle: 0.0303889
[800]	valid_0's rmsle: 0.0301221
[900]	valid_0's rmsle: 0.0299047
[1000]	valid_0's rmsle: 0.0297058
[1100]	valid_0's rmsle: 0.0295327
[1200]	valid_0's rmsle: 0.0293759
[1300]	valid_0's rmsle: 0.0292374
[1400]	valid_0's rmsle: 0.029112
[1500]	valid_0's rmsle: 0.029006
[1600]	valid_0's rmsle: 0.0289112
[1700]	valid_0's rmsle: 0.0288249
[1800]	valid_0's rmsle: 0.0287481
[1900]	valid_0's rmsle: 0.028673
[2000]	valid_0's rmsle: 0.0285984
[2100]	valid_0's rmsle: 0.0285289
[2200]	valid_0's rmsle: 0.0284741
[2300]	valid_0's rmsle: 0.0284149
[2400]	valid_0's rmsle: 0.028364
[2500]	valid_0's rmsle: 0.0283112
[2600]	valid_0's rmsle: 0.0282611
[2700]	valid_0's rmsle: 0.0282225
[2800]	valid_0's rmsle: 0.0281772


In [124]:
display(temp_num.loc[temp_num['CARD_SIDO_NM']==16])
display(temp_num.iloc[1561280:1658859,:])

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CNT,CSTMR_CNT,gap,mark
1561280,16,0,0,1,1,1,2020,4,0.000000,0.000000,0.000000,0
1561281,16,0,0,1,1,1,2020,7,0.000000,0.000000,0.000000,0
1561282,16,0,0,1,1,2,2020,4,0.001088,0.002052,-0.000965,2
1561283,16,0,0,1,1,2,2020,7,0.001106,0.002036,-0.000930,2
1561284,16,0,0,1,1,3,2020,4,0.002629,0.005836,-0.003207,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1658855,16,30,14,0,2,3,2020,7,0.000000,0.000000,0.000000,0
1658856,16,30,14,0,2,4,2020,4,0.000000,0.000000,0.000000,0
1658857,16,30,14,0,2,4,2020,7,0.000000,0.000000,0.000000,0
1658858,16,30,14,0,2,5,2020,4,0.000000,0.000000,0.000000,0


In [139]:
pred = model.predict(temp_num)
temp_num['AMT'] = pred
temp = temp_num

In [140]:
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [141]:
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [152]:
submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,8.236446e+10
1,202004,강원,골프장 운영업,5.794562e+10
2,202004,강원,과실 및 채소 소매업,2.155375e+10
3,202004,강원,관광 민예품 및 선물용품 소매업,1.641869e+10
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,2.376972e+10
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,6.101351e+09
1390,202007,충북,한식 음식점업,2.282981e+10
1391,202007,충북,호텔업,3.065684e+10


In [151]:
temp.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,8.236446e+10
1,202004,강원,골프장 운영업,5.794562e+10
2,202004,강원,과실 및 채소 소매업,2.155375e+10
3,202004,강원,관광 민예품 및 선물용품 소매업,1.641869e+10
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,2.376972e+10


In [138]:
data = pd.read_csv('201901-202003.csv')
data = data.fillna('')
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

# 검증

In [145]:
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values):
    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)

    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)

    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)

    # 평균을 낸다.
    mean_difference = difference.mean()

    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)

    return score


In [147]:
data = pd.read_csv('201901-202003.csv')
data = data.fillna('')
df = data.copy()
df = df[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
df = df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)
df = df.loc[df['REG_YYMM']==202003]
df = df[['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]

In [148]:
submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.loc[submission['REG_YYMM']==202004]
submission = submission[['CARD_SIDO_NM', 'STD_CLSS_NM']]
submission = submission.merge(df, left_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission = submission.fillna(0)
AMT = list(submission['AMT'])*2

submission = pd.read_csv('submission.csv', index_col=0)
submission['AMT'] = AMT
#submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,9.605901e+07
1,202004,강원,골프장 운영업,2.915798e+09
2,202004,강원,과실 및 채소 소매업,9.948169e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,1.331730e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00


In [150]:
rmsle(temp.AMT,submission.AMT)

9.327041829280729